In [1]:
    # Load packages for (ADV_analytics kernel)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

%matplotlib inline
print("All packages are available!")

All packages are available!


Notes:
1.  1 blank 'party_detailed' vote cast for Donald TRUMP, corrected party to REPUBLICAN pior to import.  

2.  Remaining blanks verified as writein votes cast for "OTHER" party candidates.

3.  94 samples labeled "DEMOCRAT/REPUBLICAN" in 'party_detailed':  
    utilized PA.gov to confirm HEFFLEY = REPUBLICAN, LONGIETTI = DEMOCRAT.

4.  In Nov 2020, there were four recognized political parties in Pennsylvania.  
    Democratic Party, Green Party, Libertarian Party, Republican Party.  
    DEM and REP votes accounted for 96% of total votes; "OTHER" will combine any vote for another.   

In [3]:
    # Import the first dataset
VTD_import = pd.read_csv('PA_l2_turnout_stats_block20.csv')
#print(VTD_import.head())

In [4]:
    # Inspect VTD file
    # Retain only GEOID and 03NOV2020 columns
vtd_columns = ['geoid20',
    'g20201103_reg_all', 'g20201103_voted_all', 'g20201103_pct_voted_all',
    'g20201103_reg_eur', 'g20201103_voted_eur', 'g20201103_pct_voted_eur',
    'g20201103_reg_hisp', 'g20201103_voted_hisp', 'g20201103_pct_voted_hisp',
    'g20201103_reg_aa', 'g20201103_voted_aa', 'g20201103_pct_voted_aa',
    'g20201103_reg_esa', 'g20201103_voted_esa', 'g20201103_pct_voted_esa',
    'g20201103_reg_oth', 'g20201103_voted_oth', 'g20201103_pct_voted_oth',
    'g20201103_reg_unk', 'g20201103_voted_unk', 'g20201103_pct_voted_unk']

    # Change VAR names
VTD_inspect = (VTD_import[vtd_columns]
               .rename(columns=lambda x: x.replace('g20201103_', ''))
               .rename(columns=lambda x: x.replace('pct_voted', 'turnout'))
               .rename(columns=lambda x: x.replace('_esa', '_asia'))
               .copy())

    # Create 'GEOID' column and truncate, drop original
VTD_inspect['GEOID'] = VTD_inspect['geoid20'].astype(str).str[:11]
VTD_inspect = VTD_inspect.drop(columns=['geoid20'])
                               
    # Confirm
#print(VTD_inspect.info())

In [5]:
    # Define aggregation rules
    # Use sum for all columns that do not contain 'turnout' (excluding GEOID)
sum_cols = [col for col in VTD_inspect.columns if 'turnout' not in col and col != 'GEOID']
    # Use mean for columns containing 'turnout'
mean_cols = [col for col in VTD_inspect.columns if 'turnout' in col]

    # Group by GEOID and aggregate
VTD_transform = VTD_inspect.groupby('GEOID').agg({
    **{col: 'sum' for col in sum_cols}, 
    **{col: 'mean' for col in mean_cols}}).reset_index()

    # Convert 'reg_' and 'voted_' columns to int32
convert_cols = [col for col in VTD_transform.columns if 
                col.startswith('reg_') or col.startswith('voted_')]
VTD_transform[convert_cols] = VTD_transform[convert_cols].astype('int32')

   # Confirm
#print(VTD_transform.info())
#print(VTD_transform['GEOID'].value_counts())

In [154]:
#import numpy as np

# Generate 10 random integers between 1 and 100
random_numbers = np.random.randint(1, 101, size=10)

# Calculate statistics
mean_value = np.mean(random_numbers)
median_value = np.median(random_numbers)
std_dev_value = np.std(random_numbers)

# Print results
print(f"Random Numbers: {random_numbers}")
print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")


Random Numbers: [14 97 79 25 25  4 77 90 83 56]
Mean: 55.00
Median: 66.50
Standard Deviation: 33.04


In [146]:
print(VTD_transform.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3446 entries, 0 to 3445
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   GEOID         3446 non-null   object 
 1   reg_all       3446 non-null   int32  
 2   voted_all     3446 non-null   int32  
 3   reg_eur       3446 non-null   int32  
 4   voted_eur     3446 non-null   int32  
 5   reg_hisp      3446 non-null   int32  
 6   voted_hisp    3446 non-null   int32  
 7   reg_aa        3446 non-null   int32  
 8   voted_aa      3446 non-null   int32  
 9   reg_asia      3446 non-null   int32  
 10  voted_asia    3446 non-null   int32  
 11  reg_oth       3446 non-null   int32  
 12  voted_oth     3446 non-null   int32  
 13  reg_unk       3446 non-null   int32  
 14  voted_unk     3446 non-null   int32  
 15  turnout_all   3446 non-null   float64
 16  turnout_eur   3446 non-null   float64
 17  turnout_hisp  3446 non-null   float64
 18  turnout_aa    3446 non-null 

In [51]:
    # Create "turnout_gap" for each group (turnout_grp - turnout_all)
base_cols = ['all', 'eur', 'hisp', 'aa', 'asia', 'oth', 'unk']
for grp in base_cols:
   col_name = f'turnout_{grp}'
   VTD_transform[col_name] = (VTD_transform[col_name] * 100).round(2)
for grp in base_cols[1:]:
   VTD_transform[f'turnout_{grp}'] = VTD_transform[
   f'turnout_{grp}'] - VTD_transform['turnout_all']

    # Define column order, No turnout_gap for 'all'
final_cols = ['GEOID']
for grp in base_cols:
   final_cols.extend([f'reg_{grp}', f'voted_{grp}', f'turnout_{grp}'])
   if grp != 'all':
        final_cols.append(f'turnout_gap_{grp}')

    # Reorder
VTD_tidy = VTD_transform[final_cols]
VTD_tidy['GEOID-5'] = VTD_tidy['GEOID'].astype(str).str[:5]

    # Confirm final tidy dataframe
#print(VTD_tidy.info())
#print(VTD_tidy.head())
#VTD_tidy.shape

KeyError: "['turnout_gap_eur', 'turnout_gap_hisp', 'turnout_gap_aa', 'turnout_gap_asia', 'turnout_gap_oth', 'turnout_gap_unk'] not in index"

In [ ]:
#VTD_tidy.to_excel('VTD_tidy.xlsx', index=False)

In [57]:
    # Import and tidy DHC dataset
DHC_import = pd.read_csv('pa_dhc_2020_t.csv')
#print(DHC_import.head())

In [58]:
    # Format GEOID: rename and truncate
DHC_inspect = DHC_import.rename(columns={'GEOID20': 'GEOID'})
DHC_inspect['GEOID'] = DHC_inspect['GEOID'].astype(str).str[:11]

    # Drop columns with constant entries or conflicts (County = COUNTYFP20 = GEOID)
DHC_inspect = DHC_inspect.drop(columns=[
    'STATEFP20', 'STATE', 'COUNTYFP20', 'COUNTY', 'NOT_DEF'])

    # Confirm
#print(DHC_inspect.info())
#print(DHC_inspect['GEOID'].value_counts())

In [59]:
    # Remove any record where total pop = 0, to prevent NAs in ratios
DHC_transform = DHC_inspect[(DHC_inspect['TOT_POP'] != 0) & (
                            DHC_inspect['HH_IN_HH'] != 0)].copy()

    # Compute new ratios for basic population metrics
DHC_transform['URBAN_RATIO']  = (DHC_transform['URBAN'] / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['RURAL_RATIO']  = (DHC_transform['RURAL'] / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['MALE_RATIO']   = (DHC_transform['MALE'] / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['FEMALE_RATIO'] = (DHC_transform['FEMALE'] / DHC_transform['TOT_POP'] * 100).round(2)

    # Compute new age groups
DHC_transform['F_U18']    = DHC_transform['U5_F'] + DHC_transform['5_9_F'] + \
        DHC_transform['10_14_F'] + DHC_transform['15_17_F']
DHC_transform['F_18-20s'] = DHC_transform['18_19_F'] + DHC_transform['20_F']
DHC_transform['F_30s']    = DHC_transform['30_34_F'] + DHC_transform['35_39_F']
DHC_transform['F_40s']    = DHC_transform['40_44_F'] + DHC_transform['45_49_F']
DHC_transform['F_50s']    = DHC_transform['50_54_F'] + DHC_transform['55_59_F']
DHC_transform['F_60up']    = (DHC_transform['60_61_F'] + DHC_transform['62_64_F'] + \
        DHC_transform['65_66_F'] + DHC_transform['67_69_F'] + DHC_transform['70_74_F'] + \
        DHC_transform['75_79_F'] + DHC_transform['80_84_F'] + DHC_transform['85_O_F'])
DHC_transform['M_U18']    = DHC_transform['U5_M'] + DHC_transform['5_9_M'] + \
        DHC_transform['10_14_M'] + DHC_transform['15_17_M']
DHC_transform['M_18-20s'] = DHC_transform['18_19_M'] + DHC_transform['20_M']
DHC_transform['M_30s']    = DHC_transform['30_34_M'] + DHC_transform['35_39_M']
DHC_transform['M_40s']    = DHC_transform['40_44_M'] + DHC_transform['45_49_M']
DHC_transform['M_50s']    = DHC_transform['50_54_M'] + DHC_transform['55_59_M']
DHC_transform['M_60up']    = (DHC_transform['60_61_M'] + DHC_transform['62_64_M'] + \
        DHC_transform['65_66_M'] + DHC_transform['67_69_M'] + DHC_transform['70_74_M'] + \
        DHC_transform['75_79_M'] + DHC_transform['80_84_M'] + DHC_transform['85_O_M'])

    # Compute ratios for the new age groups (relative to total FEMALE or MALE)
DHC_transform['F_U18_RATIO']    = (DHC_transform['F_U18']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['F_18-20s_RATIO'] = (DHC_transform['F_18-20s'] / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['F_30s_RATIO']    = (DHC_transform['F_30s']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['F_40s_RATIO']    = (DHC_transform['F_40s']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['F_50s_RATIO']    = (DHC_transform['F_50s']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['F_60up_RATIO']    = (DHC_transform['F_60up']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['M_U18_RATIO']    = (DHC_transform['M_U18']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['M_18-20s_RATIO'] = (DHC_transform['M_18-20s'] / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['M_30s_RATIO']    = (DHC_transform['M_30s']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['M_40s_RATIO']    = (DHC_transform['M_40s']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['M_50s_RATIO']    = (DHC_transform['M_50s']    / DHC_transform['TOT_POP'] * 100).round(2)
DHC_transform['M_60up_RATIO']    = (DHC_transform['M_60up']    / DHC_transform['TOT_POP'] * 100).round(2)

    # Define new household groups sexuality will be considered
    # Marital status not considered, assuming relationship / marriage will have same outcome
DHC_transform['O_SEX_REL'] = DHC_transform['O_SEX_MAR'] + DHC_transform['O_SEX_UMAR']
DHC_transform['S_SEX_REL'] = DHC_transform['S_SEX_MAR'] + DHC_transform['S_SEX_UMAR']
    # "OTHER" = non-related (roommates) and persons in institution quarters
DHC_transform['OTHER_HH'] = DHC_transform['OTH_NONREL'] + DHC_transform['GRP_QUART']

    # --- Compute Ratios for housing groups
DHC_transform['F_SINGLE_RATIO'] = (DHC_transform['F_ALONE'] / DHC_transform['HH_IN_HH'] * 100).round(2)
DHC_transform['M_SINGLE_RATIO'] = (DHC_transform['M_ALONE'] / DHC_transform['HH_IN_HH'] * 100).round(2)
DHC_transform['F_LED_HH_RATIO'] = (DHC_transform['F_N_ALONE'] / DHC_transform['HH_IN_HH'] * 100).round(2)
DHC_transform['M_LED_HH_RATIO'] = (DHC_transform['M_N_ALONE'] / DHC_transform['HH_IN_HH'] * 100).round(2)
DHC_transform['OTH_HH_RATIO'] = (DHC_transform['OTHER_HH'] / DHC_transform['HH_IN_HH'] * 100).round(2)
DHC_transform['OPPSEX_RATIO'] = (DHC_transform['O_SEX_REL'] / DHC_transform['HH_IN_HH'] * 100).round(2)
DHC_transform['SAMESEX_RATIO'] = (DHC_transform['S_SEX_REL'] / DHC_transform['HH_IN_HH'] * 100).round(2)

    # Confirm
#pd.set_option('display.max_columns', None)
#print(DHC_transform.head(20))

In [60]:
    # Define final column order
final_cols = [
    'GEOID', 'TOT_POP', 
    'URBAN', 'URBAN_RATIO', 
    'RURAL', 'RURAL_RATIO', 
    'MED_AGE', 'MED_AGE_M', 'MED_AGE_F', 
    'F_U18', 'F_U18_RATIO', 'M_U18', 'M_U18_RATIO',  
    'F_18-20s', 'F_18-20s_RATIO', 'M_18-20s', 'M_18-20s_RATIO',  
    'F_30s', 'F_30s_RATIO', 'M_30s', 'M_30s_RATIO',  
    'F_40s', 'F_40s_RATIO', 'M_40s', 'M_40s_RATIO',  
    'F_50s', 'F_50s_RATIO', 'M_50s', 'M_50s_RATIO',  
    'F_60up', 'F_60up_RATIO', 'M_60up', 'M_60up_RATIO',  
    'MALE', 'MALE_RATIO', 'FEMALE', 'FEMALE_RATIO',  
    'F_ALONE', 'F_SINGLE_RATIO', 'M_ALONE', 'M_SINGLE_RATIO', 
    'F_N_ALONE', 'F_LED_HH_RATIO', 'M_N_ALONE', 'M_LED_HH_RATIO',
    'O_SEX_REL', 'OPPSEX_RATIO', 'S_SEX_REL', 'SAMESEX_RATIO', 
    'OTHER_HH', 'OTH_HH_RATIO', 'HH_IN_HH']
DHC_tidy = DHC_transform[final_cols]

    # Confirm
#print(DHC_tidy.info())
#DHC_tidy.shape

In [61]:
#DHC_tidy.to_excel('DHC_tidy.xlsx', index=False)

In [62]:
    # Import third dataset
MEDSL_import = pd.read_csv('pa_medsl_20_general.csv')

    # Inspect
#print(MEDSL_import.head())

In [63]:
    # Inspect MEDSL file
    # Retain only the needed columns
MEDSL_inspect = MEDSL_import[['office', 'county_fips', 'precinct',  
                            'party_detailed', 'votes']]

    # Rename columns
MEDSL_inspect = MEDSL_inspect.rename(columns={
    'county_fips': 'GEOID-5', 
    'party_detailed': 'PARTY', 
    'votes': 'VOTES'})

    # fix GEOID type
MEDSL_inspect['GEOID-5'] = MEDSL_inspect['GEOID-5'].astype(str)

    # Confirm
#print(MEDSL_inspect.info())
#print(MEDSL_inspect['office'].value_counts(dropna=False))

In [64]:
    # Keep single race to avoid counting multiple votes per person
MEDSL_clean = MEDSL_inspect[MEDSL_inspect['office'] == 'AUDITOR GENERAL'] 
MEDSL_clean = MEDSL_clean.copy().drop(columns=['office']) # closest vote total to VTD

    # Fill any missing values and change GREEN and LIBERTARIAN to 'OTHER' 
MEDSL_clean.loc[:, 'PARTY'] = MEDSL_clean['PARTY'].fillna('OTHER')
MEDSL_clean['PARTY'] = MEDSL_clean['PARTY'].replace({'GREEN': 'OTHER', 
                                                     'LIBERTARIAN': 'OTHER'})
    # Confirm
#print(MEDSL_clean.info())
#print(MEDSL_clean['PARTY'].value_counts(dropna=False))

In [99]:
MEDSL_tranfm2 = MEDSL_transform1.copy().drop(columns=['precinct'])

    # Group duplicate precinct, sum vote totals
MEDSL_tranfm2 = (MEDSL_tranfm2
    .groupby(['GEOID-5', 'PRECINCT'], as_index=False).sum())

    # Confirm
#print(MEDSL_tranfm2.info())
#print(MEDSL_tranfm2.head(20))

In [103]:
print(MEDSL_tranfm2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GEOID-5  67 non-null     object
 1   PARTY    67 non-null     object
 2   VOTES    67 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.7+ KB
None


In [101]:
    # Rename columns
MEDSL_tranfm3 = MEDSL_tranfm2.rename(columns={
    'DEMOCRAT': 'DEM_VOTES',
    'REPUBLICAN': 'REP_VOTES',
    'OTHER': 'OTH_VOTES'})

    # Change vote columns to int32
vote_cols = ['DEM_VOTES', 'REP_VOTES', 'OTH_VOTES']
MEDSL_tranfm3[vote_cols] = MEDSL_tranfm3[vote_cols].astype('int32')

    # Confirm
#print(MEDSL_tranfm3.head())
#print(MEDSL_tranfm3.info())

KeyError: "None of [Index(['DEM_VOTES', 'REP_VOTES', 'OTH_VOTES'], dtype='object')] are in the [columns]"

In [ ]:
MEDSL_tranfm4 = MEDSL_tranfm3.copy()

    # Compute TOTAL_VOTES, drop any where the sum of all votes = 0
MEDSL_tranfm4['TOTAL_VOTES'] = MEDSL_tranfm4[vote_cols].sum(axis=1).astype('int32')
MEDSL_tranfm4 = MEDSL_tranfm4[MEDSL_tranfm4['TOTAL_VOTES'] != 0]

    # Compute shares of votes
MEDSL_tranfm4['DEM_SHARE'] = ((MEDSL_tranfm4['DEM_VOTES'] / MEDSL_tranfm4['TOTAL_VOTES'])* 100).round(2)
MEDSL_tranfm4['REP_SHARE'] = ((MEDSL_tranfm4['REP_VOTES'] / MEDSL_tranfm4['TOTAL_VOTES'])* 100).round(2)
MEDSL_tranfm4['OTH_SHARE'] = ((MEDSL_tranfm4['OTH_VOTES'] / MEDSL_tranfm4['TOTAL_VOTES'])* 100).round(2)

    # Confirm
#print(MEDSL_tranfm4.head())
#print(MEDSL_tranfm4.info())

In [ ]:
MEDSL_tranfm5 = MEDSL_tranfm4.copy()

    # Define the political leaning function
def determine_win(row):
    shares = {
        'DEM': row['DEM_SHARE'],
        'REP': row['REP_SHARE'],
        'OTH': row['OTH_SHARE']} 
      # Sort shares in descending order
    sorted_shares = sorted(shares.items(), key=lambda x: x[1], reverse=True)
      # Top party
    max_party, max_value = sorted_shares[0]  
    second_party, second_value = sorted_shares[1]
      # Difference between top two parties
    diff = round(max_value - second_value, 2)
      # Determine PARTY_WIN (Binary: DEM = 1, otherwise 0)
    party_win = 1 if max_party == 'DEM' else 0
    return party_win, diff

    # Apply function and create two new variables
MEDSL_tranfm5[['PARTY_WIN', 'PARTY_LEAD']] = MEDSL_tranfm5.apply(determine_win, axis=1).apply(pd.Series)

# Confirm output
print(MEDSL_tranfm5[['PARTY_WIN', 'PARTY_LEAD']].head(20))

In [ ]:
    # Order variables
final_cols = ['GEOID-5', 'PRECINCT', 'TOTAL_VOTES', 'DEM_VOTES', 'DEM_SHARE', 
              'REP_VOTES', 'REP_SHARE', 'OTH_VOTES', 'OTH_SHARE', 'PARTY_LEAD', 'PARTY_WIN']
MEDSL_tidy = MEDSL_tranfm5[final_cols]

    # Confirm
#print(MEDSL_tidy.info())

In [ ]:
#MEDSL_tidy.to_excel('MEDSL_tidy.xlsx', index=False)

In [130]:
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.manova import MANOVA
import statsmodels.api as sm

    # Import here for code debugging the merge, if required

VTD_tidy = pd.read_excel('VTD_tidy.xlsx')
DHC_tidy = pd.read_excel('DHC_tidy.xlsx')
MEDSL_tidy = pd.read_excel('MEDSL_tidy.xlsx')

# inspection codes

In [132]:
    # Merge first two files
MID_tidy = pd.merge(DHC_tidy, VTD_tidy, on='GEOID', how='outer')

    # Combine other and unknown voters
MID_tidy['reg_oth'] = MID_tidy['reg_other'] + MID_tidy['reg_unk']
MID_tidy['voted_oth'] = MID_tidy['voted_other'] + MID_tidy['voted_unk']
MID_tidy.drop(columns=['reg_unk', 'voted_unk', 'turnout_unk', 'turnout_gap_unk', 
                          'reg_other', 'voted_other'], inplace=True)

    # Need to adjust GEOID in order to merge with MEDSL
MID_tidy['GEOID-10'] = MID_tidy['GEOID'].astype(str).str[:-1].astype(str)

    # Confirm
print(MID_tidy.info())

KeyError: 'reg_other'

In [ ]:
    # Merge with third dataset
PA_VOTE_DF = pd.merge(MID_join, MEDSL_join, on='JOIN', how='outer')

    # Drop columns after merging
PA_VOTE_DF.drop(columns=[
    'GEOID', 'GEOID-10', 'GEOID-5_x', 'GEOID-5_y', 
    'PRECINCT', 'BIN_x', 'BIN_y'], inplace=True)

    # Confirm
pd.set_option('display.max_columns', None)
print(PA_VOTE_DF)

In [ ]:
    # Set PARTY_WIN values to 0, will rerun function to clear merging errors
PA_VOTE_DF['PARTY_WIN'] = 0

    # group together variables for sum or mean
PA_VOTE_SUM = ['TOT_POP', 'URBAN', 'RURAL', 'F_U18', 'M_U18', 'F_18-20s', 'M_18-20s', 
               'F_30s', 'M_30s', 'F_40s', 'M_40s', 'F_50s', 'M_50s', 'F_60up', 'M_60up', 
               'MALE', 'FEMALE', 'F_ALONE', 'M_ALONE', 'F_N_ALONE', 'M_N_ALONE', 
               'O_SEX_REL', 'S_SEX_REL', 'OTHER_HH', 'reg_all', 'voted_all', 
               'reg_eur', 'voted_eur', 'reg_hisp', 'voted_hisp', 'reg_aa', 'voted_aa', 
               'reg_asia', 'voted_asia', 'reg_oth', 'voted_oth',  
               'DEM_VOTES', 'REP_VOTES', 'OTH_VOTES', 'TOTAL_VOTES', 'PARTY_WIN']

PA_VOTE_MEAN = ['URBAN_RATIO', 'RURAL_RATIO', 'MED_AGE', 'MED_AGE_M', 'MED_AGE_F', 
                'F_U18_RATIO', 'M_U18_RATIO', 'F_18-20s_RATIO', 'M_18-20s_RATIO', 
                'F_30s_RATIO', 'M_30s_RATIO', 'F_40s_RATIO', 'M_40s_RATIO', 
                'F_50s_RATIO', 'M_50s_RATIO', 'F_60up_RATIO', 'M_60up_RATIO', 
                'MALE_RATIO', 'FEMALE_RATIO', 'F_SINGLE_RATIO', 'M_SINGLE_RATIO', 
                'F_LED_HH_RATIO', 'M_LED_HH_RATIO', 'HH_IN_HH', 'OPPSEX_RATIO', 'SAMESEX_RATIO', 
                'OTH_HH_RATIO', 'turnout_all', 'turnout_eur', 'turnout_gap_eur', 
                'turnout_hisp', 'turnout_gap_hisp', 'turnout_aa', 'turnout_gap_aa', 
                'turnout_asia', 'turnout_gap_asia', 'turnout_oth', 'turnout_gap_oth', 
                'DEM_SHARE', 'REP_SHARE', 'OTH_SHARE', 'PARTY_LEAD']

    # Group by JOIN and aggregate
PA_VOTE_A = PA_VOTE_DF.groupby('JOIN').agg({
    **{col: 'sum' for col in PA_VOTE_SUM if col in PA_VOTE_DF.columns}, 
    **{col: 'mean' for col in PA_VOTE_MEAN if col in PA_VOTE_DF.columns}
      }).reset_index()

    # Convert summed variables to int
PA_VOTE_A[PA_VOTE_SUM] = PA_VOTE_A[PA_VOTE_SUM].astype('int32')
PA_VOTE_A[PA_VOTE_MEAN] = PA_VOTE_A[PA_VOTE_MEAN].astype('float32').round(2)

    # Confirm
print(PA_VOTE_A.info())
#pd.set_option('display.max_columns', None)
#print(PA_VOTE_A.head(100))

In [ ]:
PA_VOTE_B = pd.read_excel('PA_VOTE_B.xlsx')

In [ ]:
    # Drop any records where TOTAL_VOTES are 0
#PA_VOTE_B = PA_VOTE_A[PA_VOTE_A['TOTAL_VOTES'] != 0].copy()

    # Identify columns that contain 'turnout_'
#columns_to_zeroize = [col for col in PA_VOTE_B.columns if 'turnout_' in col]

   # Clear all to recalculate ratios and turnouts
#PA_VOTE_B[columns_to_zeroize] = 0

print(PA_VOTE_B.head(10))

In [ ]:
    # Compute turnout as voted_ / reg_ for each column
turnout_cols = ['all', 'eur', 'hisp', 'aa', 'asia', 'oth']
for col in turnout_cols:
   voted_col = f'voted_{col}'
   reg_col = f'reg_{col}'
   turnout_col = f'turnout_{col}'

     # Calculate turnout only where registration is greater than zero to avoid division errors
   PA_VOTE_B[turnout_col] = PA_VOTE_B[voted_col] / PA_VOTE_B[reg_col]
     # Replace NaN with 0 where division is undefined
   PA_VOTE_B[turnout_col] = PA_VOTE_B[turnout_col].fillna(0)  

   # recalculate "turnout_gap" for each group (turnout_grp - turnout_all)
for grp in turnout_cols:
   col_name = f'turnout_{grp}'
   PA_VOTE_B[col_name] = (PA_VOTE_B[col_name] * 100).round(2)

for grp in turnout_cols[1:]:
   PA_VOTE_B[f'turnout_gap_{grp}'
    ] = PA_VOTE_B[f'turnout_{grp}'] - PA_VOTE_B['turnout_all']

    # Confirm
print(PA_VOTE_B.head(10))

In [ ]:
PA_VOTE_C = pd.read_excel('PA_VOTE_B.xlsx')

In [134]:
    # Round all ratio (float) columns to 2 decimal places
PA_VOTE_C = PA_VOTE_B.round({col: 2 for col in PA_VOTE_B.select_dtypes(include=['float']).columns}).copy()

    # Set JOIN as index, and drop it from numerical calculations
PA_VOTE_C = PA_VOTE_C.set_index('JOIN', drop=True)

    # Apply function again
PA_VOTE_C['PARTY_WIN'] = PA_VOTE_C.apply(lambda row: determine_win(row)[0], axis=1)

    # EDA - Correlation Heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(PA_VOTE_C.corr(), cmap='coolwarm', annot=False, linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

    # EDA - Distribution Check
PA_VOTE_C.hist(figsize=(12, 8), bins=30, edgecolor='black')
plt.suptitle('Feature Distributions')
plt.show()

NameError: name 'PA_VOTE_B' is not defined

In [ ]:
    # Identify highly correlated features (threshold = 0.85)
corr_matrix = PA_VOTE_C.corr().abs()
upper_tri = corr_matrix.where(
    ~corr_matrix.map(lambda x: x == 1).astype(bool)
).where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Features to drop due to high correlation
high_corr = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)]

    # Compute feature variances, drop very low variance
variances = PA_VOTE_C.var()
low_variance = variances[variances < 0.01].index.tolist()

    # Combine both feature sets to drop
features_to_drop = list(set(high_corr + low_variance))

    # Output list of features to drop
print(features_to_drop)

In [ ]:
    # Create a copy of the dataset for clustering
PA_VOTE_cluster = PA_VOTE_C.copy()

    # Apply transformations (inverting selected variables)
PA_VOTE_cluster['turnout_oth_inv'] = PA_VOTE_cluster['turnout_oth'] * -1
PA_VOTE_cluster['F_SINGLE_RATIO_inv'] = PA_VOTE_cluster['F_SINGLE_RATIO'] * -1
PA_VOTE_cluster['M_SINGLE_RATIO_inv'] = PA_VOTE_cluster['M_SINGLE_RATIO'] * -1
PA_VOTE_cluster['F_LED_HH_RATIO_inv'] = PA_VOTE_cluster['F_LED_HH_RATIO'] * -1

    # Define the final selected features for clustering
selected_vars = ['URBAN_RATIO', 'turnout_hisp', 'turnout_oth_inv', 'F_30s_RATIO', 'F_50s_RATIO', 
                 'SAMESEX_RATIO', 'F_SINGLE_RATIO_inv', 'M_SINGLE_RATIO_inv', 'F_LED_HH_RATIO_inv']

    # Standardize the selected features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(PA_VOTE_cluster[selected_vars])

    # Run K-Means clustering
kmeans_13 = KMeans(n_clusters=13, random_state=630, n_init=10)

    # Set
PA_VOTE_cluster[selected_vars + ['PARTY_WIN']]

PA_VOTE_cluster['Cluster_13'] = kmeans_13.fit_predict(scaled_data)

    # Inspect results
print(PA_VOTE_cluster.head())

In [ ]:
# Delete, substitute code
selected_vars = [col for col in PA_VOTE_cluster.columns if col != 'PARTY_WIN']
party_win_column = PA_VOTE_cluster['PARTY_WIN']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(PA_VOTE_cluster[selected_vars])
kmeans_13 = KMeans(n_clusters=13, random_state=630, n_init=10)
PA_VOTE_cluster['Cluster_13'] = kmeans_13.fit_predict(scaled_data)
PA_VOTE_cluster['PARTY_WIN'] = party_win_column
print(PA_VOTE_cluster.head())

In [ ]:
    # Compute distribution of PARTY_WIN across clusters
cluster_distro = (PA_VOTE_cluster.groupby('Cluster_13')['PARTY_WIN']
    .value_counts(normalize=True)
    .unstack() * 100)

    # Rename columns for clarity
cluster_distro.rename(columns={0: 'Republican (%)', 1: 'Democrat (%)'}, inplace=True)

    # Fill missing values with 0 a few will be 0
cluster_distro = cluster_distro.fillna(0)

    # Set df
PA_VOTE_13 = cluster_distro.copy()

    # Confirm
print(PA_VOTE_13)

In [ ]:
    # Extract cluster centers
cluster_centers = kmeans_13.cluster_centers_

    # Create a new df for better visualization
cluster_centers_df = pd.DataFrame(cluster_centers, columns=selected_vars)

    # Plot the cluster centers
plt.figure(figsize=(12, 6))
for i in range(len(cluster_centers)):
   plt.plot(cluster_centers_df.iloc[i], label=f'Cluster {i}')

plt.xticks(rotation=45)
plt.xlabel("Features")
plt.ylabel("Cluster Center Value")
plt.title("K-Means Cluster Centers for Each Feature")
plt.legend(title="Clusters", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

    # Create Cluster Heatmap
plt.figure(figsize=(15, 40))
sns.clustermap(pd.DataFrame(scaled_data, columns=selected_vars),
               method='average',
               metric='euclidean',
               figsize=(12, 12),
               cmap="coolwarm",
               col_cluster=False)  # Cluster only on rows

plt.title("Cluster Heatmap of Scaled Features")
plt.show()


In [ ]:
    # Create pairplot with PARTY_WIN hue
sns.pairplot(PA_VOTE_cluster, hue='PARTY_WIN', palette={0: 'red', 1: 'blue'})
plt.show()

In [ ]:
# Select significant predictors for logistic regression

initial_vars = ['URBAN_RATIO', 'turnout_hisp', 'turnout_oth_inv', 'F_30s_RATIO', 'F_50s_RATIO', 
                'SAMESEX_RATIO', 'F_SINGLE_RATIO_inv', 'M_SINGLE_RATIO_inv', 'F_LED_HH_RATIO_inv']
PA_VOTE_1 = PA_VOTE_cluster[initial_vars]
y = PA_VOTE_cluster['PARTY_WIN']

# Standardize selected features
scaler = StandardScaler()
trim_1 = scaler.fit_transform(PA_VOTE_1)

# Add constant for intercept
trim_1 = sm.add_constant(trim_1)

# Fit logistic regression model with selected features
logit_1_model = sm.Logit(y, trim_1).fit()

# Display results
logit_1_model.summary()

In [ ]:
    Pseudo R² = 0.1760 (Moderate explanatory power)
    Log-Likelihood = -434.31 (Improved from null model: -527.07)
    LLR p-value = 3.587e-35 (Highly significant overall)



In [ ]:
# Select significant predictors for logistic regression

final_vars = ['URBAN_RATIO', 'turnout_hisp', 'F_30s_RATIO', 'SAMESEX_RATIO', 'F_LED_HH_RATIO_inv']
PA_VOTE_2 = PA_VOTE_cluster[final_vars]
y = PA_VOTE_cluster['PARTY_WIN']

# Standardize selected features
scaler = StandardScaler()
trim_2 = scaler.fit_transform(PA_VOTE_2)

# Add constant for intercept
trim_2 = sm.add_constant(trim_2)

# Fit logistic regression model with selected features
logit_2_model = sm.Logit(y, trim_2).fit()

# Display results
logit_2_model.summary()

In [ ]:
Pseudo R² = 0.1750 (Slight decrease from 0.1760, but still strong)  
Log-Likelihood = -434.81 (Minimal change from -434.31)  
LLR p-value = 3.729e-37 (Highly significant, now e-02 smaller)

In [ ]:
# Generate predicted probabilities using the updated logit_2_model
pred_2_probs = logit_2_model.predict(trim_2)  # Probabilities
pred_2 = (pred_2_probs > 0.5).astype(int)  # Convert to binary predictions

# Compute accuracy
accuracy_2 = accuracy_score(y, pred_2)
report_2 = classification_report(y, pred_2)
conf_matrix_2 = confusion_matrix(y, pred_2)

# Display results
print(f'Logistic Regression Accuracy: {accuracy_2:.4f}')
print(report_2)
print('Confusion Matrix:')
print(conf_matrix_2)

In [ ]:
# Extract coefficients
coefs = logit_2_model.params[1:]  # Exclude the constant term
features = final_vars  # Use original feature names

# Plot feature importance
plt.figure(figsize=(10, 5))
plt.barh(final_vars, coefs)
plt.xlabel('Logistic Regression Coefficient')
plt.ylabel('Feature')
plt.title('Feature Importance in Predicting PARTY_WIN')
plt.axvline(x=0, color='gray', linestyle='--')  # Reference line at 0
plt.show()


In [138]:
from docx import Document

In [144]:

doc = Document()

# Title Page
doc.add_heading("Predicting Political Outcomes with Clustering & Logistic Regression", level=1)
doc.add_paragraph("ANA 630 - Final Project\nJason Noble\nDate")

# Abstract
doc.add_heading("Abstract", level=2)
doc.add_paragraph(
    "This study explores whether demographic characteristics can reliably predict party affiliation "
    "in Pennsylvania’s 2020 election. Using clustering (K-Means, Hierarchical Agglomerative Clustering) "
    "and logistic regression, we identify key predictors and evaluate their statistical significance. "
    "Feature selection, hypothesis testing, and model performance were analyzed to determine which variables "
    "had the most impact. The final logistic regression model achieved **79.5% accuracy**, confirming that factors "
    "such as urbanization, household structure, and voter turnout ratios play a role in predicting political outcomes."
)

# Introduction
doc.add_heading("Introduction", level=2)
doc.add_paragraph(
    "Objective:\n"
    "- Null Hypothesis (H₀): No demographics can predict party affiliation.\n"
    "- Alternative Hypothesis (H₁): There are specific demographic characteristics that can reliably predict party affiliation.\n"
    "- Compare different clustering strategies to identify **optimal k**.\n"
    "- Predict which party (Republican or Democrat) will win a region using clustering and logistic regression.\n"
    "- Assess model accuracy and predictive power."
)

# The Challenge of the Merge
doc.add_heading("The Challenge of the Merge", level=2)
doc.add_paragraph(
    "Each voting district begins with a **GEOID** in their 20-digit ID, so initially, joining datasets using GEOID seemed "
    "like a viable approach. However, discrepancies arose between datasets:\n"
    "- **MEDSL vote counts differ from VTD** (MEDSL is raw precinct votes before county verification).\n"
    "- **Some counties use voting precincts as Census districts, while others do not.**\n"
    "- **Without Secretary of State info, clean conversion was impossible.**\n\n"
    "Approach taken:\n"
    "- **Convert GEOID structures** (VTD.GEOID-12 and DHC.GEOID-15) to **GEOID-11**.\n"
    "- **Aggregate data** using sum or mean during merging.\n"
    "- **Assign bin labels** to categorize precincts for merging.\n"
    "- **Recalculate turnout ratios after merging.**"
)

# The Data
doc.add_heading("The Data", level=2)
doc.add_paragraph(
    "**Datasets Used:**\n"
    "1. **VTD:** L2 Voter File - 2020 Elections Turnout Statistics for Pennsylvania, aggregated to VTD level (GEOID-11).\n"
    "   - 149 variables x 336,986 records.\n"
    "2. **DHC:** U.S. Census Bureau - 2020 Select Demographic and Housing Characteristic (DHC) Data for Pennsylvania.\n"
    "   - 90 variables x 3,447 records.\n"
    "3. **MEDSL:** MIT Election Data and Science Lab - 2020 general election results for Pennsylvania, at the precinct level.\n"
    "   - 25 variables x 186,492 records."
)

# Data Preprocessing
doc.add_heading("Data Preprocessing", level=2)
doc.add_paragraph(
    "**Challenges & Efforts:**\n"
    "- MEDSL data was **filtered for November 2020 elections** and matched to one race to ensure one vote per person.\n"
    "- Missing **PARTY data was corrected** (e.g., one Trump vote labeled incorrectly as 'Other').\n"
    "- **80 join groups lacked demographic data** post-merge but still had vote tallies → These were redistributed proportionally.\n"
    "- **Feature engineering:**\n"
    "  - Combined variables into logical groups (age, housing, turnout categories).\n"
    "  - Normalized distributions where necessary.\n"
    "  - Dependent Variable: **PARTY_WIN (Republican = 0, Democrat = 1).**"
)

# Clustering Approach
doc.add_heading("Clustering Approach", level=2)
doc.add_paragraph(
    "**Methods Tested:**\n"
    "- **K-Means Clustering** and **Hierarchical Agglomerative Clustering (HAC).**\n"
    "- **Calculated WCSS** to determine optimal k (Elbow Method ~13).\n\n"
    "**Findings:**\n"
    "- **13 Clusters:** Showed clear voting patterns, but were strongly Republican.\n"
    "- **68 Clusters:** More balanced PARTY_WIN distribution, but worse overall model performance.\n"
    "- Standardized features made HAC **less effective** (variables became too uniform)."
)

# Cluster Distributions
doc.add_heading("Cluster Distributions", level=2)
doc.add_paragraph(
    "**Key Observations:**\n"
    "- **13 Clusters:** Certain clusters were **overwhelmingly Republican (>25% higher share).**\n"
    "- **Two clusters were competitive (≈50/50 split).**\n"
    "- **68 Clusters caused over-segmentation**, limiting pattern clarity."
)

# Logistic Regression Approach
doc.add_heading("Logistic Regression Approach", level=2)
doc.add_paragraph(
    "**Why Logistic Regression?**\n"
    "- Ideal for **binary outcomes** (Republican = 0, Democrat = 1).\n"
    "- Allows for **feature selection and model interpretability.**\n\n"
    "**Feature Selection Process:**\n"
    "1. **Started with full feature set** (high collinearity issues).\n"
    "2. **Removed redundant features** using Variance Inflation Factor (VIF).\n"
    "3. **Kept only statistically significant predictors (p-values < 0.05).**"
)

# Model Performance & Accuracy
doc.add_heading("Model Performance & Accuracy", level=2)
doc.add_paragraph(
    "**Final Model Performance (13 Clusters):**\n"
    "- **Accuracy: ~79.5%.**\n"
    "- **High precision for Republicans (81%).**\n"
    "- **Low recall for Democrat wins (27%) → Many misclassified as Republican.**\n\n"
    "**Confusion Matrix:**\n"
    "| Actual vs. Predicted | Predicted Republican (0) | Predicted Democrat (1) |\n"
    "|----------------------|-------------------------|------------------------|\n"
    "| **Actual Republican (0)** | **704** (True Negatives) | **31** (False Positives) |\n"
    "| **Actual Democrat (1)** | **166** (False Negatives) | **62** (True Positives) |\n"
)

# Key Findings
doc.add_heading("Key Findings", level=2)
doc.add_paragraph(
    "**Summary of Insights:**\n"
    "- **Clustering revealed distinct political patterns.**\n"
    "- **Logistic regression successfully predicted Republican wins but struggled with Democrat classification.**\n"
    "- **Feature selection improved model performance.**\n\n"
    "**Limitations & Next Steps:**\n"
    "- **Dataset imbalance** (more Republican wins than Democrats).\n"
    "- The merging process **spread Democratic votes, diluting their impact.**\n"
    "- Consider **alternative dataset joining methods.**\n"
    "- Explore **Random Forest, SVM, or ensemble models** for better classification.\n"
    "- Continue **feature engineering** to improve recall for Democrats."
)

# Conclusion & Recommendations
doc.add_heading("Conclusion & Recommendations", level=2)
doc.add_paragraph(
    "**Final Takeaways:**\n"
    "- **Clustering provided useful insights** into regional voting trends.\n"
    "- **Logistic regression worked well but needs improvement in Democrat predictions.**\n"
    "- **13 clusters proved to be the best strategy.**\n\n"
    "**Recommendations:**\n"
    "- ***Find a way to accurately merge all three datasets.***\n"
    "- **Use resampling techniques (SMOTE) to balance dataset.**\n"
    "- **Try alternative classification models for better recall.**\n"
    "- **Refine feature selection with PCA.**"
)

    # Save
docx_file_path = r"C:/Users/nobul/anaconda3/ALL_DATA/ANA630/NOBLE_ANA630_Final.docx"
doc.save(docx_file_path)

# Compute VIF for housing variables
X_housing = PA_VOTE_cluster[['F_SINGLE_RATIO_inv', 'M_SINGLE_RATIO_inv', 
                             'F_LED_HH_RATIO_inv', 'M_LED_HH_RATIO_inv']]
X_housing = sm.add_constant(X_housing)  # Add constant for proper VIF calculation

vif_data = pd.DataFrame()
vif_data['Feature'] = X_housing.columns
vif_data['VIF'] = [variance_inflation_factor(X_housing.values, i) for i in range(X_housing.shape[1])]

print(vif_data)

Some domains naturally have low intrinsic dimensionality (like voter demographics).If the dataset has inherently low dimensionality (e.g., strong correlations between features), then only a few PCs should matter.

# Check significance of individual features using ANOVA
anova_results = {}
for feature in manova_features:
    model = ols(f"{feature} ~ HAC_Cluster", data=PA_VOTE_F).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    anova_results[feature] = anova_table["PR(>F)"].iloc[0]  # Extract p-value

# Sort features by significance (lowest p-values first)
sorted_features = sorted(anova_results.items(), key=lambda x: x[1])

# Select the top 3 most significant features
top_3_features = [sorted_features[i][0] for i in range(3)]
print("Top 3 Features Selected:", top_3_features)
#Top 5 Features Selected: ['voted_all', 'OTH_VOTES', 'M_SINGLE_RATIO', 'F_SINGLE_RATIO', 'M_LED_HH_RATIO']


# Standardize the data for K-Means
X_kmeans = StandardScaler().fit_transform(PA_VOTE_C.drop(columns=['PARTY_WIN']))

# Compute WCSS for a range of clusters
wcss = []
k_range = range(1, 68)  # Testing k from 1 to 14

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=630, n_init=10)
    kmeans.fit(X_kmeans)
    wcss.append(kmeans.inertia_)  # Inertia = WCSS

# Plot the Elbow Curve
plt.figure(figsize=(8, 5))
plt.plot(k_range, wcss, marker='o', linestyle='-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS (Within-Cluster Sum of Squares)')
plt.title('Elbow Method for Optimal k')
plt.xticks(k_range)
plt.grid()
plt.show()


Clustering may not work for this dataset, with such popular cities, it does take a lot of rural areas to equal a single record from Philly or Pittsburg.

# Normalize by total votes to adjust for urban vs. rural imbalance
PA_VOTE_E['weighted_turnout'] = PA_VOTE_E['voted_all'] / PA_VOTE_E['OTH_VOTES']

# Apply log transformation to compress large variations
PA_VOTE_E['log_voted_all'] = np.log1p(PA_VOTE_E['voted_all'])
PA_VOTE_E['log_OTH_VOTES'] = np.log1p(PA_VOTE_E['OTH_VOTES'])

# Select features for clustering
features_for_clustering = ['log_voted_all', 'log_OTH_VOTES', 'M_SINGLE_RATIO', 'weighted_turnout']

# Standardize data
scaler = StandardScaler()
PA_VOTE_scaled = scaler.fit_transform(PA_VOTE_E[features_for_clustering])

# Compute linkage using Ward’s method
linkage_weighted = linkage(PA_VOTE_scaled, method='ward')

# Plot dendrogram to determine optimal cutoff
plt.figure(figsize=(12, 6))
dendrogram(linkage_weighted, truncate_mode='level', p=10)
plt.axhline(y=8, color='r', linestyle='--')  # Adjust cutoff manually
plt.title("Dendrogram with Distance Cutoff")
plt.xlabel("Data Points")
plt.ylabel("Distance")
plt.show()

# Set optimal cluster number
optimal_k = 5  # Adjust based on dendrogram
clusters_weighted = fcluster(linkage_weighted, optimal_k, criterion='maxclust')

# Add clusters to DataFrame
PA_VOTE_E['HAC_Cluster_Weighted'] = clusters_weighted

# Display cluster distribution
print(PA_VOTE_E['HAC_Cluster_Weighted'].value_counts().sort_index())

# Standardize features for hierarchical clustering
PA_VOTE_hier = StandardScaler().fit_transform(PA_VOTE_D.drop(columns=['PARTY_WIN']))

# Compute linkage matrix
linkage_matrix = linkage(PA_VOTE_hier, method='ward')

# Plot dendrogram
plt.figure(figsize=(12, 6))
dendrogram(linkage_matrix, truncate_mode='level', p=5)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Cluster Index')
plt.ylabel('Distance')
plt.show()


    # Apply PCA on the standardized dataset
pca = PCA()
PA_VOTE_PCA = pca.fit_transform(PA_VOTE_scaled)

    # Plot explained variance to determine optimal number of components
explained_variance = np.cumsum(pca.explained_variance_ratio_)

plt.figure(figsize=(8,5))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()
